In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
from sklearn.model_selection import train_test_split

import keras
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,BatchNormalization
from keras.callbacks import EarlyStopping,LearningRateScheduler,ReduceLROnPlateau
from keras.optimizers import SGD,RMSprop
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score

from keras.models import Sequential
warnings.filterwarnings('ignore')

from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.layers import Dropout

/var/folders/w1/1j9z35wd10d0sqn1dg67ymkr0000gn/T/ipykernel_15634/1149283108.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [31]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

def display_images(desired_shape, images, num_images=6):
    plt.figure(figsize=(12, 6))
    for i in range(num_images):
        ax = plt.subplot(2, 3, i + 1)
        plt.imshow(images[i].reshape(desired_shape[0], desired_shape[1]), cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

def process_pixel_data(pixel_data, desired_shape):
    return np.array([np.array([float(x) for x in item.split()]).reshape(desired_shape) for item in pixel_data])

def process_labels(labels):
    # Convert labels to numerical format if they're not numeric
    unique_labels = {label: idx for idx, label in enumerate(np.unique(labels))}
    return np.array([unique_labels[label] for label in labels]), unique_labels

def make_model(pixel_data, labels):
    # Process pixel data
    input_shape = (128, 128, 1) # Corresponds to target_size in preprocess.py, but with an added dimension
    processed_images = process_pixel_data(pixel_data, desired_shape=input_shape)

    # display_images(input_shape, processed_images)

    # Process labels
    processed_labels, label_map = process_labels(labels)
    processed_labels = to_categorical(processed_labels)

    # Splitting data into training and testing sets
    train_img, test_img, train_cancer_cat, test_cancer_cat = train_test_split(
        processed_images, processed_labels, test_size=0.2, random_state=42)
    
    # CNN Model
    model = Sequential([
        Conv2D(256, (3,3), activation='relu', padding='same', input_shape=input_shape),
        MaxPool2D(2,2),
        BatchNormalization(),
        Conv2D(64, (3,3), activation='relu', padding='same'),
        MaxPool2D(2,2),
        BatchNormalization(),
        Conv2D(32, (3,3), activation='relu', padding='same'),
        MaxPool2D(2,2),
        BatchNormalization(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(64, activation='relu'),
        Dense(len(label_map), activation='softmax') 
    ])

    model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_img, train_cancer_cat, validation_data=(test_img, test_cancer_cat), epochs=30)

    # Evaluation and reporting
    print("Training Accuracy:", history.history['accuracy'][-1])
    print("Validation Accuracy:", history.history['val_accuracy'][-1])

    # Evaluation
    print("Training accuracies:")
    print(history.history["accuracy"][-1])

    mlp_model = Sequential([
        Flatten(input_shape=input_shape),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(label_map), activation='softmax')
    ])

    mlp_model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    mlp_history = mlp_model.fit(
        train_img, train_cancer_cat,
        validation_data=(test_img, test_cancer_cat),
        epochs=30
    )

    # Evaluation and reporting
    print("Training Accuracy:", mlp_history.history['accuracy'][-1])
    print("Validation Accuracy:", mlp_history.history['val_accuracy'][-1])

    # Evaluation
    print("Training accuracies:")
    print(mlp_history.history["accuracy"][-1])

    # return model, label_map, history


In [ ]:
import os
import subprocess
!find . -name ".DS_Store" -delete

# TODO: Need to make a for-loop that goes through every folder in images and annotations. This is 
# because the way the code works, it only goes through one folder at a time, so we need a loop
# that calls big_helper.py multiple times. I don't care about runtime anymore...

# NOTE: There will be Not-Found errors because not all the image files are downloaded even though all
# the annotations are downloaded.

# # Some of the stencil code uses depricated code, but it is still needed
# import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning)

# This code block basically preprocesses the image data and puts it into a CSV file
for folder in os.listdir("dl_data/annotations"):
    dicom_path = f"dl_data/images/Lung_Dx-{folder}"
    annotation_path = f"dl_data/annotations/{folder}" 

    folder_list = ['A0001', 'A0002', 'A0003', 'A0004', 'A0005', 'B0004', 'B0005', 'E0001', 'E0002', 'G0004', 'G0005']
    
    if folder not in folder_list:
        continue
    else:
        print(folder)
    
    command = [
        "python", "preprocess.py", "--dicom-mode", "CT",
        "--dicom-path", dicom_path,
        "--annotation-path", annotation_path,
        "--classfile", "category.txt"
    ]
    
    # Execute the command
    subprocess.run(command)

In [32]:
data = pd.read_csv('output.csv')
make_model(data['pixel_data'], data['cancer_type'])

Epoch 1/30
4/4 [==============================] - 8s 2s/step - loss: 28.6765 - accuracy: 0.4904 - val_loss: 3501.5703 - val_accuracy: 0.3846
Epoch 2/30
4/4 [==============================] - 7s 1s/step - loss: 3.2530 - accuracy: 0.7981 - val_loss: 1514.5366 - val_accuracy: 0.4615
Epoch 3/30
4/4 [==============================] - 6s 1s/step - loss: 0.9902 - accuracy: 0.8750 - val_loss: 314.4474 - val_accuracy: 0.8462
Epoch 4/30
4/4 [==============================] - 6s 1s/step - loss: 1.6833 - accuracy: 0.9038 - val_loss: 559.1036 - val_accuracy: 0.6923
Epoch 5/30
4/4 [==============================] - 6s 1s/step - loss: 0.4887 - accuracy: 0.9327 - val_loss: 456.2019 - val_accuracy: 0.7692
Epoch 6/30
4/4 [==============================] - 6s 1s/step - loss: 3.2838e-05 - accuracy: 1.0000 - val_loss: 360.5824 - val_accuracy: 0.7692
Epoch 7/30
4/4 [==============================] - 6s 1s/step - loss: 2.3383e-07 - accuracy: 1.0000 - val_loss: 290.9708 - val_accuracy: 0.7692
Epoch 8/30
4/4 [

Training Accuracy: 1.0
Validation Accuracy: 0.8461538553237915
Training accuracies:
1.0
Epoch 1/30
4/4 [==============================] - 1s 74ms/step - loss: 8487.9727 - accuracy: 0.3077 - val_loss: 2191.0872 - val_accuracy: 0.7692
Epoch 2/30
4/4 [==============================] - 0s 40ms/step - loss: 10162.5400 - accuracy: 0.5673 - val_loss: 2296.6638 - val_accuracy: 0.7692
Epoch 3/30
4/4 [==============================] - 0s 40ms/step - loss: 10160.1992 - accuracy: 0.5865 - val_loss: 4287.1050 - val_accuracy: 0.7692
Epoch 4/30
4/4 [==============================] - 0s 40ms/step - loss: 9948.4219 - accuracy: 0.6346 - val_loss: 3628.7122 - val_accuracy: 0.7692
Epoch 5/30
4/4 [==============================] - 0s 39ms/step - loss: 11491.4629 - accuracy: 0.6923 - val_loss: 1226.8722 - val_accuracy: 0.8462
Epoch 6/30
4/4 [==============================] - 0s 41ms/step - loss: 8430.8330 - accuracy: 0.6154 - val_loss: 957.9778 - val_accuracy: 0.9231
Epoch 7/30
4/4 [========================